In [1]:
import pandas as pd


areas = ['binhu', 'liangxi', 'xinwu', 'huishan', 'xishan', 'jiangyinshi', 'yixingshi']

# 创建一个空字典来存储每个区域的 DataFrame
dfs = {}

# 读取每个区域的 CSV 文件，并将其存储在字典中
for area in areas:
    df = pd.read_csv(f'./Data/二手房信息{area}.csv')
    dfs[area] = df
    print(dfs[area].shape)
    # print(type(dfs[area]))  #验证不受影响

dfs['xishan'].head(5)

(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)
(29, 12)


,标题,小区,区域,总价,单价,户型,面积,朝向,装修情况,楼层类别,总层数,建筑类型
0,诚售，看房方便，边套户型。采光好。,瑞景望府,安镇,140万,"14,737元/平",3室2厅,95平米,东南,毛坯,低楼层,共26层,板楼
1,锡山实小旁 花园叠墅 房东诚售,建发玖里湾4期,安镇,420万,"31,358元/平",4室2厅,133.94平米,南,精装,高楼层,共6层,暂无数据
2,全屋空调带地暖，精装三房位置好，前排楼栋无遮挡,华润江南府北区,东亭,146万,"16,679元/平",3室2厅,87.54平米,南,精装,低楼层,共34层,板楼
3,精装小三房 保养好楼层好 看房方便,万业观山泓郡,安镇,156万,"18,263元/平",3室2厅,85.42平米,南,精装,中楼层,共33层,暂无数据
4,蓝光雍锦园荟聚地铁口 纯毛坯 带车位 诚心出售 随时看,蓝光雍锦园,东亭,138万,"12,739元/平",3室2厅,108.33平米,南,毛坯,低楼层,共33层,板楼


<h1>地区:</h1>
binhu liangxi xinwu huishan xishan jiangyinshi yixingshi  
<h1>数据集:</h1>
dfs['area']


In [2]:
from pyecharts.charts import Timeline, Bar, Map
from pyecharts import options as opts
from Switch_CH import switch_ch


houseNumAreas = Timeline(init_opts=opts.InitOpts(width="1000px", height="450px"))

for area in areas:
    values = dfs[area].区域.value_counts()
    
    c = (
        Bar(init_opts=opts.InitOpts(width="600px", height="600px"))
        .add_xaxis(values.index.tolist())
        .add_yaxis("二手房数量", values.values.tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title=switch_ch(area)))
    )
    houseNumAreas.add(c, switch_ch(area))

    
houseNumAreas.render("./HTML/houseNumAreas_bar_timeline.html")

'D:\\Code\\Pycharm\\Homework_Code\\Final_Assignment\\HTML\\houseNumAreas_bar_timeline.html'

In [3]:
values = dfs['xishan'].区域.value_counts()

unit_price_dit = {}
#求地区房价的平均值
for area in areas:
    price = dfs[area].单价.str.extract(r'(\d+,\d+)')
    price = price.dropna()
    price = price[0].str.replace(',', '').astype(int)
    print(area, int(price.mean()))
    unit_price_dit[area] = int(price.mean())
    
unit_price_dit = {switch_ch(z): v for z, v in unit_price_dit.items()}
print(unit_price_dit)
    
unitPriceMap = (
    Map(init_opts=opts.InitOpts(width="850px", height="850px"))
    .add("二手房平均单价", [list(z) for z in zip(unit_price_dit.keys(), unit_price_dit.values())], "无锡")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="无锡市二手房单价分布图"),
        visualmap_opts=opts.VisualMapOpts(
            max_= 22000,
            min_= 10000,
        )
    )
    
)

unitPriceMap.render("./HTML/unitPriceMap_map_visualmap_piecewise.html")

binhu 21552
liangxi 19003
xinwu 14761
huishan 13783
xishan 15219
jiangyinshi 11651
yixingshi 11580
{'滨湖区': 21552, '梁溪区': 19003, '新吴区': 14761, '惠山区': 13783, '锡山区': 15219, '江阴市': 11651, '宜兴市': 11580}


'D:\\Code\\Pycharm\\Homework_Code\\Final_Assignment\\HTML\\unitPriceMap_map_visualmap_piecewise.html'

In [4]:
from pyecharts.charts import Pie, Timeline

house_towards = Timeline(init_opts=opts.InitOpts(width="500px", height="500px"))
for area in areas:
    dfs[area].朝向 = dfs[area].朝向.str.replace(' ', '')
    towardsValues = dfs[area].朝向.value_counts()
    
    chart1 = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(towardsValues.index.tolist(), towardsValues.values.tolist())],
            radius=["40%", "75%"],
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=f"{switch_ch(area)}朝向分布图"),
            legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_right="2%"),
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    house_towards.add(chart1, switch_ch(area))
house_towards.render("./HTML/house_towards_pie_timeline.html")


'D:\\Code\\Pycharm\\Homework_Code\\Final_Assignment\\HTML\\house_towards_pie_timeline.html'

In [5]:
from pyecharts.charts import Timeline, Pie
from pyecharts import options as opts

fulCondition = Timeline(init_opts=opts.InitOpts(width="500px", height="500px"))

for area in areas:
    fulConditionValues = dfs[area].装修情况.value_counts()
    
    chart2 = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(fulConditionValues.index.tolist(), fulConditionValues.values.tolist())],
            radius=["30%", "75%"],
            center=["50%", "50%"],
            rosetype="radius",
        )
        .set_global_opts(title_opts=opts.TitleOpts(title=f"{switch_ch(area)}装修情况分布图"),
                         legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_right="2%"),)
        
    )
    fulCondition.add(chart2, switch_ch(area))

fulCondition.render("./HTML/fulConditionValues_raduius_timeline.html")


'D:\\Code\\Pycharm\\Homework_Code\\Final_Assignment\\HTML\\fulConditionValues_raduius_timeline.html'

In [ ]:
from pyecharts.charts import Page

page = (
    Page(layout=Page.DraggablePageLayout)
)

page.add(houseNumAreas, unitPriceMap, house_towards, fulCondition)
# page.render("./HTML/page_horizontal.html")
page.save_resize_html(source='./HTML/page_horizontal.html',cfg_file='./HTML/chart_config.json',dest='./HTML/result.html')